# Question 1


We need to use the following update equation which was derived in the project instructions. 


$$e^{\Omega(\omega)\theta}=cos(\|\omega\|\theta)\bold{I_4} + sin(\|\omega\|\theta)\frac{\Omega(\omega)} {\|\omega\|}$$


In [ ]:
import spatialmath as sm
from spatialmath.base import *
import numpy as np

asdf = sm.UnitQuaternion.Rx(np.pi/2)




def updateQuaternion(original: sm.UnitQuaternion, bodyAngularVelocity: np.ndarray, deltaT: float) -> sm.UnitQuaternion:
    """
    Update the quaternion representing the orientation of a body given its angular velocity.

    This function computes the change in the body's orientation based on the given body angular velocity and 
    the time step (deltaT), and returns the updated orientation as a quaternion.

    Parameters:
    - original (sm.UnitQuaternion): The original quaternion representing the body's orientation.
    - bodyAngularVelocity (np.ndarray): The angular velocity of the body in radians per second, represented as a 3D column vector.
    - deltaT (float): The time step over which the orientation change occurs, in seconds.

    Returns:
    - sm.UnitQuaternion: The updated quaternion representing the body's new orientation.

    """
    Omega = np.vstack((np.hstack((np.array([[0]]), -bodyAngularVelocity.T)), np.hstack((bodyAngularVelocity, -skew(bodyAngularVelocity)))))
    theta = np.acos(original.data[0][0])
    normOmega = np.linalg.norm(bodyAngularVelocity)
    exponent = np.cos(normOmega * theta) * np.eye(4) + np.sin(normOmega * theta) * Omega / normOmega
    print("Theta: " + str(exponent))
    
    omega = skew(bodyAngularVelocity)

    return original * exponent

updateQuaternion(asdf, np.array([1, 0, 0]).reshape(3,1), .1)



Theta: [[ 0.70710678 -0.70710678  0.          0.        ]
 [ 0.70710678  0.70710678  0.          0.        ]
 [ 0.          0.          0.70710678  0.70710678]
 [ 0.          0.         -0.70710678  0.70710678]]


ValueError: bad operands